In [37]:
from mypyc.ir.ops import IncRef
from pydantic import BaseModel
from typing import TypedDict,Literal


In [64]:
class PortFolioState(BaseModel):
    usd_amt: float
    total_amt: float=0
    target_currency: Literal['INR','EUR']
    total_amt: float=0
    amt_inr: float=0
    amt_eur: float=0





In [47]:
def total_amt(state: PortFolioState)->PortFolioState:
    state.total_amt=state.usd_amt*0.85
    return state

In [65]:
def amt_inr(state: PortFolioState)->PortFolioState:
    state.amt_inr=state.usd_amt*85
    return state


In [66]:
def amt_eur(state: PortFolioState)->PortFolioState:
    state.amt_eur=state.usd_amt*0.085
    return state

In [67]:
def currency_conversion(state:PortFolioState)->str:
    return state.target_currency


In [68]:
from langgraph.graph import StateGraph,START,END

builder = StateGraph(PortFolioState)
builder.add_node('total_amt',total_amt)
builder.add_node('amt_inr',amt_inr)
builder.add_node('amt_eur',amt_eur)
builder.add_edge(START,"total_amt")
builder.add_conditional_edges("total_amt",
    currency_conversion,{
        "INR": "amt_inr",
        "EUR": "amt_eur"
    }
)
builder.add_edge(["amt_inr","amt_eur"],END)
graph = builder.compile()


In [69]:
graph.invoke({"usd_amt":10000,"target_currency":"INR"})

{'usd_amt': 10000.0,
 'total_amt': 8500.0,
 'target_currency': 'INR',
 'amt_inr': 850000.0,
 'amt_eur': 0}